# Part One

First, data structures. Lets start with records. They need to be orderable, so we can sort them.

In [1]:
from dataclasses import dataclass, field
from datetime import datetime
import re

PATTERN = re.compile(r"\[(\d*)-(\d*)-(\d*) (\d*):(\d*)] (.*)") 

@dataclass(frozen=True, order=True)
class Record:
    dt: datetime = field(compare=True)
    event: str = field(compare=False)


def record_factory(record_string):
    match = PATTERN.match(record_string).groups()
    dt = [int(s) for s in match[:-1]]
    event = match[-1]
    return Record(datetime(*dt), event)

In [2]:
record_factory("[1518-11-01 00:00] Guard #10 begins shift")

Record(dt=datetime.datetime(1518, 11, 1, 0, 0), event='Guard #10 begins shift')

In [3]:
record_factory("[1518-11-01 00:00] Guard #10 begins shift") < record_factory("[1518-11-01 00:05] falls asleep")

True

Okay, now that we have an orderable Record class, lets get our input.

In [4]:
def read_input(filename):
    with open(filename) as records:
        records = [record_factory(record) for record in records.readlines()]
    return sorted(records)

In [5]:
read_input("testinput.txt")

[Record(dt=datetime.datetime(1518, 11, 1, 0, 0), event='Guard #10 begins shift'),
 Record(dt=datetime.datetime(1518, 11, 1, 0, 5), event='falls asleep'),
 Record(dt=datetime.datetime(1518, 11, 1, 0, 25), event='wakes up'),
 Record(dt=datetime.datetime(1518, 11, 1, 0, 30), event='falls asleep'),
 Record(dt=datetime.datetime(1518, 11, 1, 0, 55), event='wakes up'),
 Record(dt=datetime.datetime(1518, 11, 1, 23, 58), event='Guard #99 begins shift'),
 Record(dt=datetime.datetime(1518, 11, 2, 0, 40), event='falls asleep'),
 Record(dt=datetime.datetime(1518, 11, 2, 0, 50), event='wakes up'),
 Record(dt=datetime.datetime(1518, 11, 3, 0, 5), event='Guard #10 begins shift'),
 Record(dt=datetime.datetime(1518, 11, 3, 0, 24), event='falls asleep'),
 Record(dt=datetime.datetime(1518, 11, 3, 0, 29), event='wakes up'),
 Record(dt=datetime.datetime(1518, 11, 4, 0, 2), event='Guard #99 begins shift'),
 Record(dt=datetime.datetime(1518, 11, 4, 0, 36), event='falls asleep'),
 Record(dt=datetime.datetime(1

We now have the input in a usable form. The plan is to build a dictionary with each guard as a key, and the value being the list of that guards sleep and wake events. This can then be used to calculate the total time each guard spends asleep. The data can then be sorted by time, (ignoring the date), and used to find the overlap.

First organising the data on each guard:

In [6]:
def get_guard_data(records):
    guard_id_pattern = re.compile(r"Guard #(\d*).*")
    data = {}
    cur_guard = None
    for record in records:
        match = guard_id_pattern.match(record.event)
        if match:
            cur_guard = int(match.group(1))
        else:
            data.setdefault(cur_guard, []).append(record)
    return data

In [7]:
test_guard_data = get_guard_data(read_input("testinput.txt"))

Next, get total sleep time. `datetime.timedelta` is helpful for this. It doesn't record minutes in the delta though, so a conversion from seconds is necessary. The pairs function helps by giving each sleep/wake pair as a tuple.

In [8]:
def pairs(seq):
    "s0, s1, s2, s3 -> (s0, s1), (s2, s3)"
    return zip(seq[::2], seq[1::2])

In [9]:
from datetime import timedelta


def total_sleep_time(guard_data):
    guards = {}
    for guard_id in guard_data:
        total = timedelta()
        sleep_periods = pairs(guard_data[guard_id])
        for period in sleep_periods:
            total += period[1].dt - period[0].dt
        guards[guard_id] = total.seconds//60
    return guards

In [10]:
print(total_sleep_time(get_guard_data(read_input("testinput.txt"))))


{10: 50, 99: 30}


Now a function to find the minute when the guard was asleep most. By sorting the records, scanning through them, and recording the maximum overlap depth, we should find the correct time; this assumes there is only one minute that is correct.

In [11]:
from itertools import groupby
def find_most_common_minute(records):
    points = groupby(sorted(records, key=lambda r: r.dt.time()), key=lambda r: r.dt.minute)
    depth = 0
    max_depth = 0
    max_depth_min = None
    for k, point_set in points:
        for point in point_set:
            if point.event == 'falls asleep':
                depth += 1
            elif point.event == 'wakes up':
                depth -= 1

        if depth > max_depth:
            max_depth = depth
            max_depth_min = point.dt.minute
    return max_depth_min, max_depth

In [12]:
print(find_most_common_minute(test_guard_data[10]))

(24, 2)


I had to solve a subtle bug with this function. Consider this visualisation:

```
Date   ID   Minute
            000000000011111111112222222222333333333344444444445555555555
            012345678901234567890123456789012345678901234567890123456789
11-01  #10  ...####...#########.........................................
11-03  #10  ........##..................................................
```

Notice that at 00:10 on 1518-11-01 the guard falls asleep, but at the same time on 1518-11-03 the guard wakes up. The function `find_most_common_minute()` originally would give an incorrect output at this point. The points are sorted by time, but in the case of a conflict the original order is preserved, and so earlier dates come first (from `read_input()`). Therefore the record for the 'falls asleep' event on 11-01 is reached first; the original function would then increment `depth` and find this was a new maximum. However, because the guard had woken up on 11-03 at the same time, actually the depth shouldn't change at all. Therefore I changed the function to run through all of the records with a common minute, only updating the `max_depth` afterwards.

So far everything works for the test data. Lets try it on the actual input.

In [13]:
def solution1(filename):
    records = read_input(filename)
    guard_data = get_guard_data(records)
    totals = total_sleep_time(guard_data)
    max_total = max(totals.items(), key=lambda i: i[1])
    mcm = find_most_common_minute(guard_data[max_total[0]])[0]
    print(max_total, mcm)
    print(max_total[0] * mcm)

In [14]:
solution1("testinput.txt")

(10, 50) 24
240


In [15]:
solution1("input.txt")

(1993, 513) 36
71748


# Part Two

Now we need to find the guard that is most frequently asleep on the same minute, along with that minute. I think the way to solve this is to run `find_most_common_minute` on each guard, and find from this output what the max of each frequency is.

In [16]:
def solution2(filename):
    records = read_input(filename)
    guard_data = get_guard_data(records)
    mcm = {}
    for guard in guard_data:
        mcm[guard] = find_most_common_minute(guard_data[guard])
    max_mcm_depth = max(mcm, key=lambda k: mcm[k][1])
    print(max_mcm_depth, mcm[max_mcm_depth])
    print(max_mcm_depth * mcm[max_mcm_depth][0])

In [17]:
solution2("testinput.txt")

99 (45, 3)
4455


In [18]:
solution2("input.txt")

2137 (50, 19)
106850


# Visualiser

Lets visualise the guards schedules, in a similar way to the problem description.

In [19]:
def vis_header(id_width=2):
    header = "Date".ljust(len('XX-XX') + 2) + "ID".ljust(id_width + 3) + "Minute"
    lines = [header]
    
    date_id_width = len(header) - len('Minute')
    lines.append(' '*date_id_width + ''.join([str(ten_min) * 10 for ten_min in range(6)]))
    
    min_line = '0123456789' * 6
    lines.append(' ' * date_id_width + min_line)
    
    return lines

def vis_lines(guard, records, id_width=2):
    data_lines = {}
    for period in pairs(records):
        p = (period[0].dt, period[1].dt)
        data_lines.setdefault((p[0].month, p[0].day),
                                  [
                                      f"{p[0].month:0>2}-{p[0].day:0>2}",
                                      f"#{guard:0>{id_width}}", 
                                      ['.']*60
                                  ]
                             )[2][p[0].minute:p[1].minute] = ['#']*(p[1].minute - p[0].minute)
    
    
    lines = []
    for l in data_lines.values():
        l[2] = ''.join(l[2])
        lines.append('  '.join(l))
    return lines
    
def visualiser(guard_data, key=None):
    id_width = max([len(str(k)) for k in guard_data.keys()])
    
    lines = []
    lines.extend(vis_header(id_width))
    
    data_lines = []
    for guard in guard_data:
        data_lines.extend(vis_lines(guard, guard_data[guard], id_width))

    lines.extend(sorted(data_lines, key=key))
    
    return '\n'.join(lines)
        

In [20]:
print(visualiser(test_guard_data))

Date   ID   Minute
            000000000011111111112222222222333333333344444444445555555555
            012345678901234567890123456789012345678901234567890123456789
11-01  #10  .....####################.....#########################.....
11-02  #99  ........................................##########..........
11-03  #10  ........................#####...............................
11-04  #99  ....................................##########..............
11-05  #99  .............................................##########.....


In [21]:
def by_id(s):
    return re.match(r"\d{2}-\d{2}  #(\d*)", s).group(1)
print(visualiser(test_guard_data, key=by_id))

Date   ID   Minute
            000000000011111111112222222222333333333344444444445555555555
            012345678901234567890123456789012345678901234567890123456789
11-01  #10  .....####################.....#########################.....
11-03  #10  ........................#####...............................
11-02  #99  ........................................##########..........
11-04  #99  ....................................##########..............
11-05  #99  .............................................##########.....


In [24]:
def highlighter(vis, minute):
    vis = vis.split('\n')
    gap = len(vis[1]) - len(vis[1].strip())
    for i in range(3, len(vis)):
        if vis[i][gap+minute] == '#':
            vis[i] = vis[i][:gap+minute] + 'O' + vis[i][gap+minute+1:]
    return '\n'.join(vis)

In [25]:
guard_data = get_guard_data(read_input("input.txt"))
vis = visualiser({2137: guard_data[2137]})
vis = highlighter(highlighter(vis, 51), 50)
print(vis)

Date   ID     Minute
              000000000011111111112222222222333333333344444444445555555555
              012345678901234567890123456789012345678901234567890123456789
02-20  #2137  ..........................................########OO######..
02-25  #2137  ..................................................OO###.....
02-27  #2137  ..................................................OO#######.
03-01  #2137  .............#####################################OO........
03-06  #2137  ................................................##OO###.....
03-22  #2137  .....................#################.............O#.......
03-23  #2137  .................###############################............
03-27  #2137  ......####.....................................###OO#######.
03-29  #2137  ................................................##OO#...##..
04-14  #2137  .........#########......##########################OO##......
04-21  #2137  ...............................................###OO........
04-2